### Exploratory Data Analysis (EDA)
Collect data, and use RESTful API and web scraping. Convert the data into a dataframe and perform data wrangling. You will perform exploratory data analysis and data visualization skills.

Determine, based on the data for the first stage, whether the second stage will land. 

#### Exploratory analysis using SQL

In [1]:
import sys
!{sys.executable} -m pip install sqlalchemy==1.4 && pip install sqlalchemy==1.3.24
!{sys.executable} -m pip install ipython-sql

  Using cached SQLAlchemy-1.3.24-cp38-cp38-win_amd64.whl (1.2 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.0
    Uninstalling SQLAlchemy-1.4.0:
      Successfully uninstalled SQLAlchemy-1.4.0


In [2]:
import sqlalchemy
%load_ext sql
%sql sqlite:///my_data1.db

In [3]:
import csv, sqlite3

# my_data1 is the name of file
con = sqlite3.connect('my_data1.db')
cur = con.cursor()

In [4]:
import pandas as pd

# convert CSV to dataframe
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")

# convert dataframe to sql database
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

101

Display the names of the unique launch sites in the space mission

In [5]:
%%sql
select distinct Launch_Site
from SPACEXTBL

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


Display 5 records where launch sites begin with the string 'CCA'

In [6]:
%%sql
select *
from SPACEXTBL where Launch_Site like '%CCA%' limit 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


Display the total payload mass carried by booster launched by NASA (CRS)

In [7]:
%%sql
select sum(PAYLOAD_MASS__KG_)
from SPACEXTBL where Customer like '%NASA%'

 * sqlite:///my_data1.db
Done.


sum(PAYLOAD_MASS__KG_)
107010


Display average payload mass carried by booster version F9 v1.1

In [8]:
%%sql
select avg(PAYLOAD_MASS__KG_)
from SPACEXTBL where Booster_Version like '%F9 v1.1%'

 * sqlite:///my_data1.db
Done.


avg(PAYLOAD_MASS__KG_)
2534.6666666666665


List the date when the first succesful landing outcome in ground pad was achieved

In [9]:
%%sql
select min(substr(Date, 7, 4)) as Year, Date
from SPACEXTBL where `Landing _Outcome` like '%Success (ground pad)%' 

 * sqlite:///my_data1.db
Done.


Year,Date
2015,22-12-2015


List the names of the boosters which have successfully landed on drone ship and have payload mass greater than 4000 but less than 6000

In [10]:
%%sql
select Booster_Version
from (
select *
from SPACEXTBL where `Landing _Outcome` like '%Success (drone ship)%'
) 
where PAYLOAD_MASS__KG_ between 4000 and 6000

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


List the total number of successful and failure mission outcomes

In [11]:
%%sql
select count(*) as Successful_Missions, 
(select count(*) from SPACEXTBL) - (select count(*) from SPACEXTBL where Mission_Outcome like '%Success%') as Unsuccessful_Missions
from SPACEXTBL where Mission_Outcome like '%Success%'

 * sqlite:///my_data1.db
Done.


Successful_Missions,Unsuccessful_Missions
100,1


List the names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [12]:
%%sql
select Booster_Version 
from SPACEXTBL where Payload_Mass__kg_ = (select max(Payload_Mass__kg_) from SPACEXTBL)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


List the records which will display the months, failure landing_outcomes in drone ship, booster versions, launch_site for year 2015

In [13]:
%%sql
select substr(Date, 4, 2) as Month, `Landing _Outcome`, Booster_Version, Launch_Site
from SPACEXTBL where Date like '%2015%'

 * sqlite:///my_data1.db
Done.


Month,Landing _Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
02,Controlled (ocean),F9 v1.1 B1013,CCAFS LC-40
03,No attempt,F9 v1.1 B1014,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40
04,No attempt,F9 v1.1 B1016,CCAFS LC-40
06,Precluded (drone ship),F9 v1.1 B1018,CCAFS LC-40
12,Success (ground pad),F9 FT B1019,CCAFS LC-40


List the failed landing outcomes in drone ship, their booster versions, and launch site names for year 2015

In [14]:
%%sql
select Booster_Version, Launch_Site
from (
select *
from SPACEXTBL where `Landing _Outcome` like '%Failure (drone ship)%'
)
where substr(Date, 7, 4) like  '%2015%'

 * sqlite:///my_data1.db
Done.


Booster_Version,Launch_Site
F9 v1.1 B1012,CCAFS LC-40
F9 v1.1 B1015,CCAFS LC-40


Rank the count of successful landing_outcomes beween the date 04-06-2010 and 20-03-2017 in descending order

In [15]:
%%sql
select `Landing _Outcome` as Landing_Outcome, count(*) as Count
from SPACEXTBL where substr(Date, 7, 4) between '2010' and '2017' 
group by Landing_Outcome order by Count desc

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Count
Success (drone ship),12
No attempt,12
Success (ground pad),8
Failure (drone ship),5
Controlled (ocean),4
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
